In [1]:
import pandas as pd

df_train = pd.read_csv('../data/train_data_stg.csv', delimiter='|')
df_test = pd.read_csv('../data/test_data_stg.csv', delimiter='|')

df = pd.concat([df_train, df_test], ignore_index=True)
df = df.drop(['rank'], axis=1)

In [2]:
import numpy as np

df = np.log(df)

In [3]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.25, random_state=42)

In [4]:
import numpy as np
import tensorflow.keras.layers as layers

X_train = np.array(train_data['t'])
normalizer = layers.Normalization(input_shape=[1,], axis=None)
normalizer.adapt(X_train)

2024-07-02 18:06:05.606711: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-02 18:06:05.609627: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-02 18:06:05.618466: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 18:06:05.635555: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 18:06:05.635579: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-02 18:06:05.647665: I tensorflow/core/platform/cpu_feature_guard.cc:

In [5]:
import tensorflow as tf

model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=64, activation='relu'),
    layers.Dense(units=64, activation='relu'),
    layers.Dense(units = 1)
])

In [6]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 1)              │             3 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,356 (17.02 KB)

 Trainable params: 4,353 (17.00 KB)

 Non-trainable params: 3 (16.00 B)

In [7]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mean_absolute_error'
)

In [8]:
history = model.fit(
    train_data['t'],
    train_data['p'],
    epochs=100,
    validation_split=0.2
)

Epoch 1/100


6087/6087 ━━━━━━━━━━━━━━━━━━━━ 6s 871us/step - loss: 1.6205 - val_loss: 1.1038
Epoch 2/100
6087/6087 ━━━━━━━━━━━━━━━━━━━━ 5s 885us/step - loss: 1.1108 - val_loss: 1.1267
Epoch 3/100
6087/6087 ━━━━━━━━━━━━━━━━━━━━ 5s 860us/step - loss: 1.1119 - val_loss: 1.1040
Epoch 4/100
6087/6087 ━━━━━━━━━━━━━━━━━━━━ 5s 854us/step - loss: 1.1085 - val_loss: 1.1214
Epoch 5/100
6087/6087 ━━━━━━━━━━━━━━━━━━━━ 5s 858us/step - loss: 1.1103 - val_loss: 1.1032
Epoch 6/100
6087/6087 ━━━━━━━━━━━━━━━━━━━━ 5s 852us/step - loss: 1.1105 - val_loss: 1.1063
Epoch 7/100
6087/6087 ━━━━━━━━━━━━━━━━━━━━ 5s 854us/step - loss: 1.1100 - val_loss: 1.1035
Epoch 8/100
6087/6087 ━━━━━━━━━━━━━━━━━━━━ 5s 849us/step - loss: 1.1110 - val_loss: 1.1370
Epoch 9/100
6087/6087 ━━━━━━━━━━━━━━━━━━━━ 6s 943us/step - loss: 1.1097 - val_loss: 1.1042
Epoch 10/100
6087/6087 ━━━━━━━━━━━━━━━━━━━━ 5s 850us/step - loss: 1.1101 - val_loss: 1.1038
Epoch 11/100
6087/6087 ━━━━━━━━━━━━━━━━━━━━ 5s 852us/step - loss: 1.1086 - val_loss: 1.1058
Epoch 12/

In [9]:
prediction = model.predict(test_data['t'])

   1/2536 ━━━━━━━━━━━━━━━━━━━━ 1:36 38ms/step

2536/2536 ━━━━━━━━━━━━━━━━━━━━ 2s 603us/step


In [10]:
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error

print("Mean Absolute Error (MAE): %.2f" % mean_absolute_error(test_data[['p']], prediction))
print("Root Mean Squarred Error (RMSE): %.2f" % root_mean_squared_error(test_data[['p']], prediction))
print("R2 Score %.2f" % r2_score(test_data[['p']], prediction))

Mean Absolute Error (MAE): 1.10
Root Mean Squarred Error (RMSE): 1.46
R2 Score 0.46
